# Event Creator from JSON (Refactored)

This notebook automates the creation and management of Google Calendar events based on a JSONC schedule file (`calendars/ysc/ysc_cal.jsonc`). It leverages centralized utilities for consistency.

## Configuration and Setup

In [1]:
import os
import sys
import json
import datetime
import pandas as pd

# Add src to path so we can import our modules
sys.path.append('src')

from utils_calendar_general import (
    get_google_services, 
    load_jsonc, 
    fetch_calendar_events, 
    create_calendar_event, 
    delete_events_from_csv
)

# --- Configuration ---
CALENDAR_JSON = 'calendars/ysc/ysc_cal.jsonc'
CALENDAR_ID = 'b45a2d5121fed950d815cfa167dd4b3a6aa74c5d62fea928702e6f4300d96545@group.calendar.google.com'
CREATED_EVENTS_CSV = 'logs/created_events.csv'
SCOPES = [
    'https://www.googleapis.com/auth/spreadsheets.readonly', 
    'https://www.googleapis.com/auth/calendar.events'
]

# Credentials Paths
TOKEN_PATH = '.credentials/token.json'
CREDS_PATH = '.credentials/credentials.json'

# Testing
TEST_MODE = False

## Authentication
Initialize the Google Calendar service.

In [2]:
_, calendar_service = get_google_services(TOKEN_PATH, CREDS_PATH, SCOPES)
print("Calendar service initialized.")

Calendar service initialized.


## 1. Read Calendar JSONC
Parse the `ysc_cal.jsonc` file into a list of event objects.

In [6]:
calendar_data = load_jsonc(CALENDAR_JSON)
print(f"Loaded {len(calendar_data)} events from {CALENDAR_JSON}")

Loaded 29 events from calendars/ysc/ysc_cal.jsonc


## 2. Fetch Existing Events
Pull existing events from the calendar for comparison to avoid duplicates.

In [3]:
existing_events = fetch_calendar_events(calendar_service, CALENDAR_ID)
print(f"Found {len(existing_events)} existing events in the calendar.")

Found 214 existing events in the calendar.


Update Events for Attendance Records

In [5]:
from src.utils_calendar_general import write_events_to_csv
write_events_to_csv(existing_events, 'logs/events_attendance.csv')

Exported 214 events to logs/events_attendance.csv


## 3. Export Existing Events to CSV
(Optional) Export the list of existing events for auditing.

In [4]:
def export_existing_to_csv(events, filename='logs/existing_events.csv'):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    data = []
    for ev in events:
        data.append({
            'summary': ev.get('summary'),
            'start': ev.get('start', {}).get('dateTime'),
            'end': ev.get('end', {}).get('dateTime'),
            'id': ev.get('id')
        })
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f"Exported existing events to {filename}")

export_existing_to_csv(existing_events)

Exported existing events to logs/existing_events.csv


## 4. Identify and Create New Events
Compare the JSON data with existing events and create missing ones.

In [17]:
def sync_events(json_data, existing_events, service, calendar_id, test_mode=True):
    # Create a set of (summary, start_time) for fast lookup
    existing_set = set()
    for ev in existing_events:
        summary = ev.get('summary')
        start = ev.get('start', {}).get('dateTime')
        if start:
            existing_set.add((summary, start))

    new_events = []
    for ev_data in json_data:
        summary = ev_data.get('summary')
        start = ev_data.get('start', {}).get('dateTime')
        
        # Check if already exists
        if (summary, start) not in existing_set:
            new_events.append(ev_data)

    print(f"Identified {len(new_events)} events to create.")
    
    if test_mode:
        print(f"[TEST_MODE] Limiting creation to the first 3 events.")
        new_events = new_events[:3]

    if not new_events:
        print("No new events to create.")
        return []

    created_records = []
    for ev in new_events:
        print(f"Creating: {ev.get('summary')} at {ev.get('start', {}).get('dateTime')}")
        try:
            result = create_calendar_event(service, calendar_id, ev)
            created_records.append({
                'summary': result.get('summary'),
                'id': result.get('id'),
                'htmlLink': result.get('htmlLink')
            })
        except Exception as e:
            print(f"Failed to create event: {e}")
            
    return created_records

created_events = sync_events(calendar_data, existing_events, calendar_service, CALENDAR_ID, test_mode=TEST_MODE)

Identified 29 events to create.
Creating: Tea & Chat on Zoom (book discussion) at 2026-01-11T16:00:00
Creating: Listening Session w/ Dr. Larry Ward at 2026-01-19T15:00:00
Creating: Resource & Refuge (centering practice + reflections) at 2026-01-26T18:30:00
Creating: Tea & Chat (book discussion) at 2026-02-08T16:00:00
Creating: Film Screening: Mulberry Tree at 2026-02-16T14:00:00
Creating: Resource & Refuge (centering practice + reflections) at 2026-02-23T18:30:00
Creating: Tea & Chat (book discussion) at 2026-03-08T16:00:00
Creating: Listening Session TBD at 2026-03-16T14:30:00
Creating: Resource & Refuge (centering practice + reflections) at 2026-03-23T18:30:00
Creating: Listening Session w/ Kaira Jewel Lingo at 2026-04-13T15:30:00
Creating: Tea & Chat (book discussion) at 2026-04-19T16:00:00
Creating: Resource & Refuge (centering practice + reflections) at 2026-05-04T18:30:00
Creating: Tea & Chat (book discussion) at 2026-05-17T16:00:00
Creating: Resource & Refuge (centering practice

## 5. Save Created Events to CSV
Log IDs of created events for potential deletion later.

In [9]:
if created_events:
    df_created = pd.DataFrame(created_events)
    # Rename id to ID to match deletion expectations across notebooks
    df_created = df_created.rename(columns={'id': 'ID'})
    if os.path.exists(CREATED_EVENTS_CSV):
        df_created.to_csv(CREATED_EVENTS_CSV, mode='a', header=False, index=False)
    else:
        df_created.to_csv(CREATED_EVENTS_CSV, index=False)
    print(f"Saved {len(created_events)} records to {CREATED_EVENTS_CSV}")
else:
    print("No new records to save.")

Saved 3 records to logs/created_events.csv


## 6. Deletion Logic
Delete events from the calendar based on the `logs/created_events.csv` file.

In [ ]:
# To delete events, uncomment and run the line below:
# delete_events_from_csv(CREATED_EVENTS_CSV, calendar_service, CALENDAR_ID)